<a href="https://colab.research.google.com/github/abnerlins/Erosivity-for-Brasil/blob/main/EROSIVITY_OPTIMIZED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import warnings
import math
from matplotlib import pyplot as plt
from datetime import datetime
import sys
import numpy as np
from google.colab import drive
# Monta o Google Drive
drive.mount('/content/drive')
import warnings
# Ignora todos os warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
states = ['TO']
years = [2016]

# general path
general_path = '/content/drive/My Drive/MESTRADO/Results_Cemaden'
mit_int=360


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)
#LISTA DE HQ

df_cem_hq=pd.read_csv(general_path+'/Estacoes_Alta_Qualidade_da_Analise_Visual.csv',sep=';')

In [ ]:
for state in states:
    for year in years:
        start_time = datetime.now()
        mit=pd.Timedelta(mit_int, unit ='m')
        name_file= state+ '_'+ str(year)+'.h5'
        name_col= state+ '_ '+ str(year)
        print(f"Começando : {name_col} {start_time}")
        df_rainfall = pd.read_hdf(general_path +'/1 - Organized data/'+ name_file,'table_data')
        data = {"gauge_code": [],"EI30_T": [],"E_D": []}
        for code in df_cem_hq[name_col][:-1].dropna():
            start_time_code = datetime.now()
            name=code +'_' + str(mit_int)
            df_rainfall_events = pd.read_hdf(general_path + '/3 - Rainfall events/'+ name_file, name)
            df_partial_result = pd.DataFrame(columns=['rain_mm','duration', 'I30', 'RF_KE', 'EI30'])
            for evento in range(0,len(df_rainfall_events)):
                beg=df_rainfall_events.iloc[evento,1]
                end=df_rainfall_events.iloc[evento,1]+pd.Timedelta(hours=df_rainfall_events.iloc[evento,3])
                df_gauge_rainfall=df_rainfall[df_rainfall['gauge_code']==code]
                df_event = df_gauge_rainfall[(df_gauge_rainfall['datetime'] >= beg) & (df_gauge_rainfall['datetime'] <= end)].reset_index(drop=True)
                for i in df_event.index:
                    RF=df_event['rain_mm'][i] #mm
                    RF_intensity=(RF/10)*60 #mm/h  ###tdel
                    #kinectic energy
                    if RF==0:KE=0
                    else:
                        KE=0.119+0.0873*math.log(RF_intensity,10)  #KE= 1099*(1-0.72*math.exp(-2.088*RF_intensity)) ##MJ/ha.mm
                        RF_KE=KE*RF#MJ/ha
                        if (KE > 0.283) :KE= 0.283

                        tbeg=df_event.iloc[i,3] #df_temp é para os 30 min de análise para o i30
                        tend=df_event.iloc[i,3]+pd.Timedelta(hours=0.5)
                        df_temp=df_event[(df_event['datetime'] >= tbeg) & (df_event['datetime'] < tend)]#### < ou <=
                        I30=df_temp['rain_mm'].sum()/0.5 #mm/h #intervalo de 0,5 hrs

                        df_event.at[i,'Rf_intensity']=RF_intensity
                        df_event.at[i,'RF_KE']=RF_KE
                        df_event.at[i,'I30']=I30

                #calc ei30
                EI30=df_event['RF_KE'].sum()*df_event['I30'].max()
                #set partial results
                df_partial_result.at[evento,'rain_mm']=df_event['rain_mm'].sum()
                df_partial_result.at[evento,'duration']=df_rainfall_events.iloc[evento,3]
                df_partial_result.at[evento,'I30']=df_event['I30'].max()
                df_partial_result.at[evento,'RF_KE']=df_event['RF_KE'].sum()
                df_partial_result.at[evento,'EI30']=EI30

            #set final results
            EI30_T=df_partial_result['EI30'].sum()
            E_D=EI30_T/df_partial_result['rain_mm'].sum()
            data["gauge_code"].append(code)
            data["EI30_T"].append(EI30_T)
            data["E_D"].append(E_D)


            end_time = datetime.now()  # Fim do tempo de iteração
            print(f"  -code: {code} , tempo: {(end_time-start_time_code).total_seconds():.2f}s , {len(data['E_D'])}/{len(df_cem_hq[name_col][:-1].dropna())}")
            if  ((end_time-start_time).total_seconds()>3600)&(len(data['E_D'])%20==0):
                df_name_col = pd.DataFrame(data)
                df_name_col.to_excel(general_path+f'/Results_{name_col}p2.xlsx', index=False)
                print('backup_save')

        df_name_col = pd.DataFrame(data)
        df_name_col.to_excel(general_path+f'/Results_Cemaden/Results_{name_col}p2.xlsx', index=False)
        minutes, seconds = divmod((end_time-start_time).total_seconds(), 60)
        print(f"Finalizado: {name_col}, Duração_total: {int(minutes)}m {seconds:.2f}s")

Começando : TO_ 2014 2025-01-13 06:25:58.381567
Finalizado: TO_ 2014, Duração_total: -85m 40.06s
Começando : TO_ 2015 2025-01-13 06:25:58.704182
Finalizado: TO_ 2015, Duração_total: -85m 39.73s
Começando : TO_ 2016 2025-01-13 06:26:02.301165
  -code: 170215801A , tempo: 8.84s , 1/5
  -code: 170730601A , tempo: 3.52s , 2/5
  -code: 171830301A , tempo: 3.25s , 3/5
  -code: 171886501A , tempo: 4.40s , 4/5
  -code: 172020001A , tempo: 3.82s , 5/5
Finalizado: TO_ 2016, Duração_total: 0m 24.87s
Começando : TO_ 2017 2025-01-13 06:26:27.201752
  -code: 170215801A , tempo: 3.77s , 1/2
  -code: 171830301A , tempo: 3.84s , 2/2
Finalizado: TO_ 2017, Duração_total: 0m 8.02s
Começando : TO_ 2018 2025-01-13 06:26:35.268727
  -code: 171886501A , tempo: 5.51s , 1/1
Finalizado: TO_ 2018, Duração_total: 0m 6.33s
Começando : TO_ 2019 2025-01-13 06:26:41.632468
Finalizado: TO_ 2019, Duração_total: -1m 59.97s
Começando : TO_ 2020 2025-01-13 06:26:42.974646
  -code: 170210901A , tempo: 4.61s , 1/6
  -code: 1

In [ ]:
df_name_col = pd.DataFrame(data)
df_name_col.to_excel(general_path+f'/Results_Cemaden/Results_{name_col}.xlsx', index=False)